In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
dataset_path = "/content/drive/My Drive/MYDataset/Empty/"
# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load Training Data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load Validation Data
val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 726 images belonging to 2 classes.
Found 181 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D

# Load MobileNetV2 (pre-trained on ImageNet)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model (do not train pre-trained layers)
base_model.trainable = False

# Add Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensions
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation="sigmoid")(x)  # Binary classification (Good vs Damaged)

# Define the Model
# The error was here: you were passing the Dense layer object 'x'
# instead of the output tensor of that layer.
# Call the 'x' layer with the previous output to get the final output tensor.
model = Model(inputs=base_model.input, outputs=x)

# Compile the Model
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Model Summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
epochs = 10

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 341s 15s/step - accuracy: 0.6167 - loss: 0.6692 - val_accuracy: 0.8232 - val_loss: 0.3789
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 145s 4s/step - accuracy: 0.8023 - loss: 0.4000 - val_accuracy: 0.8287 - val_loss: 0.3828
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - accuracy: 0.8327 - loss: 0.3550 - val_accuracy: 0.8729 - val_loss: 0.3193
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.8494 - loss: 0.3328 - val_accuracy: 0.8564 - val_loss: 0.3452
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.8595 - loss: 0.3042 - val_accuracy: 0.8564 - val_loss: 0.3010
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 98s 4s/step - accuracy: 0.8573 - loss: 0.3103 - val_accuracy: 0.8619 - val_loss: 0.2814
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 97s 4s/step - accuracy: 0.8701 - loss: 0.2854 - val_accuracy: 0.8564 - val_loss: 0.2925
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - accuracy: 0.8676 - loss: 0.2847 - val_accuracy: 0.8729 - val_lo

In [ ]:
import cv2
import numpy as np
import os
import json
from matplotlib import pyplot as plt
# Import fpdf after you install it
from fpdf import FPDF

# The Rest of your Code

# Function to classify the damage size (small or big)
def classify_damage_size(bbox):
    x, y, w, h = bbox
    damage_area = w * h  # Area of the bounding box
    if damage_area < 1000:  # Threshold for small damage, adjust based on your images
        return "Small Damage"
    else:
        return "Big Damage"

# Function to detect wagons, classify damage, and label the wagons
def label_wagons_and_generate_report(image_path, output_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return

    # Resize the image to match the input size of your trained model
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)

    # Use the trained model to classify the image (good or damaged)
    prediction = model.predict(img_expanded)[0][0]

    # Use thresholding to detect potential damage areas (for counting wagons)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)

    # Find contours of detected areas
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    total_wagons = 0
    small_damage_count = 0
    big_damage_count = 0
    labeled_image = img.copy()

    # For storing wagon numbers and damage size labels
    wagon_labels = []

    for i, contour in enumerate(contours):
        if cv2.contourArea(contour) > 500:  # Threshold for minimum size of the contour
            x, y, w, h = cv2.boundingRect(contour)
            damage_size = classify_damage_size((x, y, w, h))

            # Labeling the wagons with a number and damage size
            total_wagons += 1
            wagon_number = f"#{total_wagons}"

            # Draw the bounding box and add text with the wagon number and damage size
            cv2.rectangle(labeled_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(labeled_image, f"{wagon_number} - {damage_size}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Counting small or big damage
            if damage_size == "Small Damage":
                small_damage_count += 1
            else:
                big_damage_count += 1

    # Save the labeled image
    output_image_path = os.path.join(output_path, os.path.basename(image_path))
    cv2.imwrite(output_image_path, labeled_image)

    # Generate PDF report
    pdf = FPDF()
    pdf.add_page()

    # Title of the report
    pdf.set_font("Arial", size=16, style='B')
    pdf.cell(200, 10, txt="Wagon Damage Report", ln=True, align='C')

    # Add wagon count information
    pdf.set_font("Arial", size=12)
    pdf.ln(10)  # Line break
    pdf.cell(200, 10, txt=f"Total Wagons: {total_wagons}", ln=True)
    pdf.cell(200, 10, txt=f"Small Damages: {small_damage_count}", ln=True)
    pdf.cell(200, 10, txt=f"Big Damages: {big_damage_count}", ln=True)

    # Include the image in the PDF
    pdf.ln(10)
    pdf.image(output_image_path, x=10, y=pdf.get_y(), w=180)

    # Save the PDF report
    pdf_output_path = os.path.join(output_path, "wagon_damage_report.pdf")
    pdf.output(pdf_output_path)

    print(f"Image saved at {output_image_path}")
    print(f"PDF report saved at {pdf_output_path}")
    print(f"Total Wagons: {total_wagons}, Small Damages: {small_damage_count}, Big Damages: {big_damage_count}")

    return pdf_output_path

# Example usage
image_path = "/content/drive/MyDrive/MYDataset/Empty/damage_wagons/frame_10175.jpg"
output_path = "/content/drive/MyDrive/MYDataset/Empty/damage_wagons"  # Your desired output directory
pdf_report = label_wagons_and_generate_report(image_path, output_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image saved at /content/drive/MyDrive/MYDataset/Empty/damage_wagons/frame_10175.jpg
PDF report saved at /content/drive/MyDrive/MYDataset/Empty/damage_wagons/wagon_damage_report.pdf
Total Wagons: 1, Small Damages: 0, Big Damages: 1


In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=e55e78a32a13158ecf0945aa9d77bec824f653688743e8c29a72f4485834c54e
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
